In [25]:
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
df_fraud_all = pd.read_csv("data/Fraud_Detection_sql.csv")

In [27]:
df_fraud_all.head(2)

,TRANS_DATE_TRANS_TIME,CATEGORY,AMT,GENDER,CITY,STATE,CITY_POP,JOB,DOB,IS_FRAUD
0,2019-01-01 00:00:00,misc_net,4.97,F,Moravian Falls,NC,3495,"Psychologist, counselling",1988-03-09,False
1,2019-01-01 00:00:00,grocery_pos,107.23,F,Orient,WA,149,Special educational needs teacher,1978-06-21,False


In [28]:
df_fraud_all["CATEGORY"].value_counts()

gas_transport     162800
grocery_pos       152459
home              151923
shopping_pos      144144
kids_pets         140096
shopping_net      120678
entertainment     116085
food_dining       113309
personal_care     112825
health_fitness    106036
misc_pos           99066
misc_net           78449
grocery_net        56145
travel             50279
Name: CATEGORY, dtype: int64

In [29]:
def column_labeling(DF, col_name):
    count = 0
    label_dict = {}
    column_list = []
    
    for i in DF[col_name]:
        if i in label_dict.keys():
            column_list.append(label_dict[i])
        else:
            label_dict[i] = count;
            count = count + 1
            column_list.append(label_dict[i])
            
    save_dict = pd.DataFrame()
    save_dict["string"] = label_dict.keys()
    save_dict["integer"] = label_dict.values()
    save_dict.to_csv("data/"+col_name+"_label.csv", index=False)
    
    DF[col_name] = column_list
    return DF

In [30]:
df_fraud_all = column_labeling(df_fraud_all, "CATEGORY")
df_fraud_all = column_labeling(df_fraud_all, "GENDER")
df_fraud_all = column_labeling(df_fraud_all, "STATE")
df_fraud_all.head(10)

,TRANS_DATE_TRANS_TIME,CATEGORY,AMT,GENDER,CITY,STATE,CITY_POP,JOB,DOB,IS_FRAUD
0,2019-01-01 00:00:00,0,4.97,0,Moravian Falls,0,3495,"Psychologist, counselling",1988-03-09,False
1,2019-01-01 00:00:00,1,107.23,0,Orient,1,149,Special educational needs teacher,1978-06-21,False
2,2019-01-01 00:00:00,2,220.11,1,Malad City,2,4154,Nature conservation officer,1962-01-19,False
3,2019-01-01 00:01:00,3,45.00,1,Boulder,3,1939,Patent attorney,1967-01-12,False
4,2019-01-01 00:03:00,4,41.96,1,Doe Hill,4,99,Dance movement psychotherapist,1986-03-28,False
5,2019-01-01 00:04:00,3,94.63,0,Dublin,5,2158,Transport planner,1961-06-19,False
6,2019-01-01 00:04:00,5,44.54,0,Holcomb,6,2691,Arboriculturist,1993-08-16,False
7,2019-01-01 00:05:00,3,71.65,1,Edinburg,4,6018,"Designer, multimedia",1947-08-21,False
8,2019-01-01 00:05:00,4,4.27,0,Manor,5,1472,Public affairs consultant,1941-03-07,False
9,2019-01-01 00:06:00,1,198.39,0,Clarksville,7,151785,Pathologist,1974-03-28,False


In [31]:
df_fraud_all.to_csv('data/Fraud_Detection_sql_labeled.csv', index=False)

In [32]:
df_load_test = pd.read_csv("data/Fraud_Detection_sql_labeled.csv")
df_load_test.info()
df_load_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1604294 entries, 0 to 1604293
Data columns (total 10 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   TRANS_DATE_TRANS_TIME  1604294 non-null  object 
 1   CATEGORY               1604294 non-null  int64  
 2   AMT                    1604294 non-null  float64
 3   GENDER                 1604294 non-null  int64  
 4   CITY                   1604294 non-null  object 
 5   STATE                  1604294 non-null  int64  
 6   CITY_POP               1604294 non-null  int64  
 7   JOB                    1604294 non-null  object 
 8   DOB                    1604294 non-null  object 
 9   IS_FRAUD               1604294 non-null  bool   
dtypes: bool(1), float64(1), int64(4), object(4)
memory usage: 111.7+ MB


,TRANS_DATE_TRANS_TIME,CATEGORY,AMT,GENDER,CITY,STATE,CITY_POP,JOB,DOB,IS_FRAUD
0,2019-01-01 00:00:00,0,4.97,0,Moravian Falls,0,3495,"Psychologist, counselling",1988-03-09,False
1,2019-01-01 00:00:00,1,107.23,0,Orient,1,149,Special educational needs teacher,1978-06-21,False
2,2019-01-01 00:00:00,2,220.11,1,Malad City,2,4154,Nature conservation officer,1962-01-19,False
3,2019-01-01 00:01:00,3,45.00,1,Boulder,3,1939,Patent attorney,1967-01-12,False
4,2019-01-01 00:03:00,4,41.96,1,Doe Hill,4,99,Dance movement psychotherapist,1986-03-28,False
